# Building heights in Washington, DC

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Read buildings data

#### Load shapefile directly from S3 with coordinate system set to [Maryland State Plane](https://octo.dc.gov/page/coordinate-system-standards) for distance calculations

In [4]:
gdf_src = gpd.read_file(
    "https://grid-news-static-files.s3.us-east-2.amazonaws.com/data/dc_building_heights/dc_buildings.zip",
)

#### Sort by hieght (in meters). The tallest building is the Washington Monument (168 meters, or roughly 555 feet)

In [5]:
gdf = gdf_src.sort_values("house_heig", ascending=False)

#### Get the centroid of each building

In [6]:
gdf["centroid"] = gdf["geometry"].centroid

#### Clean up dataframe

In [7]:
gdf = gdf.rename(
    columns={"egid": "id", "house_heig": "height", "altitude_m": "altitude"}
)

In [8]:
gdf = gdf.drop(
    ["cartodb_id", "slope_mean", "slope_max", "slope_min", "surface__1", "surface_to"],
    axis=1,
)

---

## City center

#### Get DC's [geometric center](https://geohack.toolforge.org/geohack.php?pagename=List_of_geographic_centers_of_the_United_States&params=38_54_15_N_77_00_58_W_region:US-DC&title=approximate+center+of+District+of+Columbia) projected in an appropriate planar coordinate system from S3

In [9]:
center_gdf = gpd.read_file(
    "https://grid-news-static-files.s3.us-east-2.amazonaws.com/data/dc_building_heights/dc_centroid_2248.geojson"
)

#### Calculate distance between each building centroid and the center of DC

In [10]:
gdf["miles"] = ((gdf.geometry.apply(lambda g: center_gdf.distance(g))) / 5280).round(3)

In [11]:
gdf["feet"] = ((gdf.geometry.apply(lambda g: center_gdf.distance(g)))).round(3)

#### Is there a relationship between height and proximity to the center? 

In [13]:
gdf[["height", "miles"]].corr()

,height,miles
height,1.000000,-0.129008
miles,-0.129008,1.000000


---

## Export

#### Save buildings file with distances with your local path (careful of Github's size limits)

In [14]:
# gdf.to_file('')